In [1]:
#Enivornment Setup

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-value-prediction-challenge/sample_submission.csv
/kaggle/input/santander-value-prediction-challenge/test.csv
/kaggle/input/santander-value-prediction-challenge/train.csv


In [3]:
#Data Read, Data Visualization,EDA Analysis,Data Pre-Processing,Data Splitting

In [4]:

# Read train and test files
train_df = pd.read_csv('/kaggle/input/santander-value-prediction-challenge/train.csv')
test_df = pd.read_csv('/kaggle/input/santander-value-prediction-challenge/test.csv')

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 4992 entries, ID to 9fc776466
dtypes: float64(4991), object(1)
memory usage: 1.8+ GB


In [6]:
# check and remove constant columns
colsToRemove = []
for col in train_df.columns:
    if col != 'ID' and col != 'target':
        if train_df[col].std() == 0: 
            colsToRemove.append(col)
        
# remove constant columns in the training set
train_df.drop(colsToRemove, axis=1, inplace=True)

# remove constant columns in the test set
test_df.drop(colsToRemove, axis=1, inplace=True) 

print("Removed `{}` Constant Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Removed `256` Constant Columns

['d5308d8bc', 'c330f1a67', 'eeac16933', '7df8788e8', '5b91580ee', '6f29fbbc7', '46dafc868', 'ae41a98b6', 'f416800e9', '6d07828ca', '7ac332a1d', '70ee7950a', '833b35a7c', '2f9969eab', '8b1372217', '68322788b', '2288ac1a6', 'dc7f76962', '467044c26', '39ebfbfd9', '9a5ff8c23', 'f6fac27c8', '664e2800e', 'ae28689a2', 'd87dcac58', '4065efbb6', 'f944d9d43', 'c2c4491d5', 'a4346e2e2', '1af366d4f', 'cfff5b7c8', 'da215e99e', '5acd26139', '9be9c6cef', '1210d0271', '21b0a54cb', 'da35e792b', '754c502dd', '0b346adbd', '0f196b049', 'b603ed95d', '2a50e001c', '1e81432e7', '10350ea43', '3c7c7e24c', '7585fce2a', '64d036163', 'f25d9935c', 'd98484125', '95c85e227', '9a5273600', '746cdb817', '6377a6293', '7d944fb0c', '87eb21c50', '5ea313a8c', '0987a65a1', '2fb7c2443', 'f5dde409b', '1ae50d4c3', '2b21cd7d8', '0db8a9272', '804d8b55b', '76f135fa6', '7d7182143', 'f88e61ae6', '378ed28e0', 'ca4ba131e', '1352ddae5', '2b601ad67', '6e42ff7c7', '22196a84c', '0e410eb3d', '992e6d1d3', '90a7

In [ ]:
#Remove Duplicate Columns


In [7]:
%%time
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cs = frame[v].columns
        vs = frame[v]
        lcs = len(cs)

        for i in range(lcs):
            ia = vs.iloc[:,i].values
            for j in range(i+1, lcs):
                ja = vs.iloc[:,j].values
                if np.array_equal(ia, ja):
                    dups.append(cs[i])
                    break

    return dups

colsToRemove = duplicate_columns(train_df)
print(colsToRemove)

['34ceb0081', '8d57e2749', '168b3e5bc', 'a765da8bc', 'acc5b709d']
CPU times: user 7min 49s, sys: 127 ms, total: 7min 49s
Wall time: 7min 49s


In [8]:
print("Removed `{}` Duplicate Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Removed `5` Duplicate Columns

['34ceb0081', '8d57e2749', '168b3e5bc', 'a765da8bc', 'acc5b709d']


In [9]:
df=colsToRemove.copy()

In [24]:
#Plotting data 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import seaborn as sns

In [10]:
def drop_sparse(train, test):
    flist = [x for x in train.columns if not x in ['ID','target']]
    for f in flist:
        if len(np.unique(train[f]))<2:
            train.drop(f, axis=1, inplace=True)
            test.drop(f, axis=1, inplace=True)
    return train, test

In [11]:
%%time
train_df, test_df = drop_sparse(train_df, test_df)

CPU times: user 698 ms, sys: 1 ms, total: 699 ms
Wall time: 697 ms


In [13]:

print("Train set size: {}".format(train_df.shape))
print("Test set size: {}".format(test_df.shape))

Train set size: (4459, 4737)
Test set size: (49342, 4736)


In [14]:
X = train_df.drop(["ID", "target"], axis=1)
y = np.log1p(train_df["target"].values)

X_test = test_df.drop(["ID"], axis=1)

In [15]:
from sklearn.model_selection import  train_test_split, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [16]:
dev_X, val_X, dev_y, val_y = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [18]:
dev_X

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
1268,0.0,0,0.0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2803,0.0,0,0.0,0,0,0,0,0,44000000.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2114,0.0,0,0.0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3150,0.0,0,0.0,0,0,0,0,0,90000.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2594,0.0,0,0.0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,436000.0,588000,0,300000,0,0,0,2572000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,0.0,0,0.0,0,0,0,0,0,4600000.0,0.0,...,0.0,0.0,4840000.0,1100000,0,0,0,0,0,0
466,0.0,0,0.0,0,0,0,0,0,24000000.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3092,0.0,0,0.0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3772,0.0,0,0.0,0,0,0,0,0,4000000.0,0.0,...,65280000.0,820000.0,0.0,0,0,0,3420000,0,0,0


In [17]:
dev_y

array([14.91412318, 17.1296966 , 16.81124288, ..., 15.89495222,
       12.89922233, 14.8087627 ])

In [29]:
from catboost import CatBoostRegressor

In [30]:
cb_model = CatBoostRegressor(iterations=2000,
                             learning_rate=0.03,
                             depth=9,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [31]:
cb_model.fit(dev_X, dev_y,
             eval_set=(val_X, val_y),
             use_best_model=True,
             verbose=50)

0:	learn: 1.7565729	test: 1.6906086	best: 1.6906086 (0)	total: 2.77s	remaining: 1h 32m 23s
50:	learn: 1.5703546	test: 1.5689399	best: 1.5689399 (50)	total: 52.3s	remaining: 33m 18s
100:	learn: 1.4698945	test: 1.5156035	best: 1.5156035 (100)	total: 1m 42s	remaining: 32m 13s
150:	learn: 1.4151539	test: 1.4953886	best: 1.4953886 (150)	total: 2m 33s	remaining: 31m 20s
200:	learn: 1.3759616	test: 1.4851163	best: 1.4851163 (200)	total: 3m 24s	remaining: 30m 26s
250:	learn: 1.3404086	test: 1.4741693	best: 1.4741693 (250)	total: 4m 14s	remaining: 29m 32s
300:	learn: 1.2977004	test: 1.4650188	best: 1.4650188 (300)	total: 5m 4s	remaining: 28m 38s
350:	learn: 1.2602958	test: 1.4594791	best: 1.4594092 (349)	total: 5m 55s	remaining: 27m 48s
400:	learn: 1.2221881	test: 1.4544874	best: 1.4544874 (400)	total: 6m 46s	remaining: 27m 1s
450:	learn: 1.1846886	test: 1.4488743	best: 1.4488743 (450)	total: 7m 38s	remaining: 26m 14s
500:	learn: 1.1535064	test: 1.4461721	best: 1.4461645 (499)	total: 8m 30s	rem

In [33]:
pred_test_cat = np.expm1(cb_model.predict(X_test))

In [34]:
sub_cat = pd.DataFrame()
sub_cat["target"] = pred_test_cat

In [ ]:
#sub["target"] = (x["target"] * 0.5 + y["target"] * 0.3 + sub_cat["target"] * 0.2) for multiple predictior

In [36]:
print(sub_cat.head(n=20))
sub_cat.to_csv('sub_cat.csv', index=False)

         target
0  1.552813e+06
1  1.825390e+06
2  2.085682e+06
3  4.033331e+06
4  1.848306e+06
